In [1]:
import pandas as pd

/Users/misha/.pyenv/versions/3.10.0/lib/python3.10/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


### Lets look at some rows...

In [2]:
df = pd.read_csv("Restaurant_Grades.csv")
df.head(3)

,DBA,BORO,BUILDING,STREET,ZIPCODE,CUISINE DESCRIPTION,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE
0,SOFRA HOMEMADE FOOD,Brooklyn,2122,86 STREET,11214.0,Turkish,Hand washing facility not provided in or near ...,Critical,19,B,06/05/2013,01/24/2022
1,SOFRA HOMEMADE FOOD,Brooklyn,2122,86 STREET,11214.0,Turkish,Non-food contact surface improperly constructe...,Not Critical,19,B,06/05/2013,01/24/2022
2,SOFRA HOMEMADE FOOD,Brooklyn,2122,86 STREET,11214.0,Turkish,Cold food item held above 41º F (smoked fish a...,Critical,19,B,06/05/2013,01/24/2022


In [3]:
# Replace spaces with underscores
df.columns = df.columns.str.replace("CUISINE DESCRIPTION","CUISINE")
df.columns = df.columns.str.replace(" ","_")
# Convert dates to datetime
df.GRADE_DATE = pd.to_datetime(df.GRADE_DATE, format="%m/%d/%Y")
df.RECORD_DATE = pd.to_datetime(df.RECORD_DATE, format="%m/%d/%Y")

In [4]:
df.value_counts()

DBA                    BORO       BUILDING  STREET                  ZIPCODE  CUISINE             VIOLATION_DESCRIPTION                                                                                                                                                                                                                                                            CRITICAL_FLAG  SCORE  GRADE  GRADE_DATE  RECORD_DATE
RADIO CITY MUSIC HALL  Manhattan  1260      AVENUE OF THE AMERICAS  10020.0  American            Non-food contact surface improperly constructed. Unacceptable material used. Non-food contact surface or equipment improperly maintained and/or not properly sealed, raised, spaced or movable to allow accessibility for cleaning on all sides, above and underneath the unit.  Not Critical   2      A      2018-11-09  2022-01-24     4
CAFE GITANE            Manhattan  242       MOTT STREET             10012.0  Moroccan            Proper sanitization not provided for utensil ware wa

### Let's zoom into 2021

In [6]:
this_year = df[df['GRADE_DATE'].dt.year == 2021]
this_year.value_counts()

DBA                              BORO       BUILDING  STREET            ZIPCODE  CUISINE                         VIOLATION_DESCRIPTION                                                                                                                                                                                                                                                            CRITICAL_FLAG  SCORE  GRADE  GRADE_DATE  RECORD_DATE
LA PALAPA TACOS                  Manhattan  230       PARK AVENUE       10169.0  Tex-Mex                         Non-food contact surface improperly constructed. Unacceptable material used. Non-food contact surface or equipment improperly maintained and/or not properly sealed, raised, spaced or movable to allow accessibility for cleaning on all sides, above and underneath the unit.  Not Critical   9      A      2021-08-23  2022-01-24     3
GO CUPS                          Manhattan  1838      2 AVENUE          10128.0  Juice, Smoothies, Fruit Salads  Non-

### It seems like restaurants have more than 1 inspections, let's deal with that

In [7]:
this_year["KEY"] = this_year[['DBA', 'BUILDING', "STREET", "ZIPCODE"]].astype(str).apply(lambda x: ' '.join(x), axis=1)
print("num ratings: {} num unique restaurants: {}".format(len(this_year), len(this_year.KEY.unique())))

num ratings: 10052 num unique restaurants: 4073


/var/folders/7j/5h5yf6k54nz1ys4trym9bls40000gn/T/ipykernel_32098/3272442714.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_year["KEY"] = this_year[['DBA', 'BUILDING', "STREET", "ZIPCODE"]].astype(str).apply(lambda x: ' '.join(x), axis=1)


### We can now either use the most recent score or an average of all the scores, let's try both

In [9]:
fscore = this_year.groupby("SCORE").SCORE.agg(["count"])
fscore["perc"] = fscore / fscore.sum() * 100

fscore_recent = this_year.drop_duplicates("KEY").groupby("SCORE").GRADE.agg(["count"])
fscore_recent["perc"] = fscore_recent / fscore_recent.sum() * 100

fscore = fscore.join(fscore_recent,lsuffix=" all", rsuffix=" most recent")#
fscore.style.format("{:,.2f}")

,count all,perc all,count most recent,perc most recent
SCORE,,,,
0,46.00,0.46,37.00,0.91
2,280.00,2.79,279.00,6.85
3,171.00,1.70,112.00,2.75
4,227.00,2.26,115.00,2.82
5,387.00,3.85,265.00,6.51
6,145.00,1.44,65.00,1.60
7,595.00,5.92,337.00,8.27
8,378.00,3.76,168.00,4.12
9,844.00,8.40,352.00,8.64


### I used the table above to make my graph on Datawrapper!!